In [1]:
# in a directory of relfectance images in flight
# for each image in the directory
#    find mean, meadian, std of image
#    extract incoming solar irradiance, timestamp, lat, lon, alt, band_name
# convert pandas dataframe to csv file


In [3]:
from os import listdir, mkdir
from os.path import isfile, join, basename, split, sep, exists
import micasense.metadata as metadata
import micasense.sequoiautils as msutils
from skimage import io
import numpy as np
import pandas as pd
import logging
import sys
import yaml
from glob import glob
import multiprocessing

logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)  # was WARNING
logging.basicConfig(format='%(asctime)s %(message)s', datefmt='%m/%d/%Y %I:%M:%S %p')

def anlayze_one_flight(flight_path):
    # get images names from a specific folder

    config_file_name = "pipeline_settings.yaml"
    with open(config_file_name, 'r') as stream:
        pipeline_config = yaml.safe_load(stream)

    flight_path = flight_path.replace(sep, '/')
    reflectance_path = join(flight_path, 'reflectance')
    raw_data_path = join(flight_path, 'rawdata')
    analysis_path = join(flight_path, 'analysis')
    analysis_path = analysis_path.replace(sep, '/')
    flight_name = basename(split(flight_path)[0])
    logging.info(flight_path)
    analysis_path_name = join(analysis_path, flight_name +'_analysis.csv')
    # if analysis file exists stop the loop
    if not exists(analysis_path):

        tif_file_names = [f for f in listdir(reflectance_path) if isfile(join(reflectance_path, f)) and f.endswith('.TIF')]
        band_name= {'Green':'GRE', 'NIR': 'NIR', 'Red':'RED','Red edge':"REG"}
        image_stats = dict()
        image_stats['folder'] = []
        image_stats['file'] = []
        image_stats['reflectance_mean'] = []
        image_stats['reflectance_std'] = []
        image_stats['reflectance_median'] = []
        image_stats['sun_irradiance(units?)'] = []
        image_stats['date_time'] = []
        image_stats['lat(deg)'] = []
        image_stats['lon(deg)'] = []
        image_stats['alt(m)'] = []
        image_stats['band_name'] = []

        for i, tif_file in enumerate(tif_file_names):


            tif_raw_data_file_path_name = join(raw_data_path, tif_file)
            tif_reflect_file_path_name = join(reflectance_path, tif_file)
            meta = metadata.Metadata(tif_raw_data_file_path_name, exiftoolPath=pipeline_config['exiftoolPath'])
            image = io.imread(tif_reflect_file_path_name)

            image_stats['folder'].append(reflectance_path)
            image_stats['file'].append(tif_file)
            image_stats['reflectance_mean'].append(np.mean(image))
            image_stats['reflectance_std'].append(np.std(image))
            image_stats['reflectance_median'].append(np.median(image))
            image_stats['sun_irradiance(units?)'].append(msutils.GetSunIrradiance(meta))
            image_stats['date_time'].append(meta.utc_time().strftime("%Y-%m-%d %H:%M:%S"))
            image_stats['lat(deg)'].append(meta.position()[0])
            image_stats['lon(deg)'].append(meta.position()[1])
            image_stats['alt(m)'].append(meta.position()[2])
            image_stats['band_name'].append(band_name[meta.band_name()])
            logging.info('Analyzing image: ' + tif_file)

        image_stats_pd = pd.DataFrame(image_stats)
        try:
            mkdir(analysis_path)
        except:
            logging.info('could not create directory ' + analysis_path)

        image_stats_pd.to_csv(analysis_path_name)
        logging.info('saving ' + flight_name +'_analysis.csv')
    else:
        logging.info(analysis_path + ' already exists... skipping this file.')

def analyze_all_flights():

    pool = multiprocessing.Pool()
    config_file_name = "pipeline_settings.yaml"
    with open(config_file_name, 'r') as stream:
        pipeline_config = yaml.safe_load(stream)

    flight_paths = glob(pipeline_config['root_folder'] + '*/')
    logging.info(flight_paths)
    #pool.map(anlayze_one_flight, flight_paths)
    for flight_path in flight_paths:
        anlayze_one_flight(flight_path)


In [4]:
analyze_all_flights()

INFO:['D:/Rafael/20181031 Ketura/problematic/Testing_dir\\20181024_1645\\', 'D:/Rafael/20181031 Ketura/problematic/Testing_dir\\20181027_1200\\']
INFO:D:/Rafael/20181031 Ketura/problematic/Testing_dir/20181024_1645/
INFO:D:/Rafael/20181031 Ketura/problematic/Testing_dir/20181024_1645/analysis already exists... skipping this file.
INFO:D:/Rafael/20181031 Ketura/problematic/Testing_dir/20181027_1200/
INFO:D:/Rafael/20181031 Ketura/problematic/Testing_dir/20181027_1200/analysis already exists... skipping this file.


In [7]:

config_file_name = "pipeline_settings.yaml"
with open(config_file_name, 'r') as stream:
    pipeline_config = yaml.safe_load(stream)

flight_paths = glob(pipeline_config['root_folder'] + '*/')

(flight_path for flight_path in flight_paths)

<generator object <genexpr> at 0x000001F91BDC0BF8>

In [8]:
file_name_path ='D:/Rafael/20181031 Ketura/problematic/Testing_dir\\20181024_1645\\'

In [9]:
basename(split(file_name_path[1])[0])

''

In [15]:
basename(split(file_name_path)[0])

'20181024_1645'